## Case 2: Fuzzy Logic Smart Home Lighting Control System

This notebook implements a fuzzy logic system to control smart home lighting based on two inputs: 
- Time of Day
- Occupancy Status

The system adjusts the lighting intensity to optimize comfort and energy efficiency.

In [8]:
# Import necessary libraries for fuzzy logic

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [9]:
# Define Fuzzy Variables
time_of_day = ctrl.Antecedent(np.arange(0, 24, 1), 'time_of_day')
occupancy = ctrl.Antecedent(np.arange(0, 2, 1), 'occupancy')
lighting_intensity = ctrl.Consequent(np.arange(0, 101, 1), 'lighting_intensity')

# Membership Functions

# for time of day - split night into late night and early morning
time_of_day['morning'] = fuzz.trapmf(time_of_day.universe, [6, 7, 9, 12])
time_of_day['afternoon'] = fuzz.trapmf(time_of_day.universe, [12, 13, 15, 18])
time_of_day['evening'] = fuzz.trapmf(time_of_day.universe, [18, 19, 20, 21])
time_of_day['late_night'] = fuzz.trapmf(time_of_day.universe, [21, 22, 23, 24])
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 1, 5, 6])

# for occupancy
occupancy['unoccupied'] = fuzz.trimf(occupancy.universe, [0, 0, 1])
occupancy['occupied'] = fuzz.trimf(occupancy.universe, [0, 1, 1])

# for lighting intensity
lighting_intensity['off'] = fuzz.trimf(lighting_intensity.universe, [0, 0, 25])
lighting_intensity['low'] = fuzz.trimf(lighting_intensity.universe, [0, 25, 50])
lighting_intensity['medium'] = fuzz.trimf(lighting_intensity.universe, [25, 50, 75])
lighting_intensity['high'] = fuzz.trimf(lighting_intensity.universe, [50, 75, 100])



### Fuzzy Rules
The system applies fuzzy rules to determine the lighting intensity based on the time of day and whether the room is occupied.

In [13]:
# Define Fuzzy Rules

# Rule 1: If it's morning and the room is occupied, lighting should be medium
rule1 = ctrl.Rule(time_of_day['morning'] & occupancy['occupied'], lighting_intensity['medium'])

# Rule 2: If it's afternoon and the room is occupied, lighting should be low
rule2 = ctrl.Rule(time_of_day['afternoon'] & occupancy['occupied'], lighting_intensity['low'])

# Rule 3: If it's evening and the room is occupied, lighting should be high
rule3 = ctrl.Rule(time_of_day['evening'] & occupancy['occupied'], lighting_intensity['high'])

# Rule 4: If it's late night and the room is unoccupied, lighting should be off
rule4 = ctrl.Rule(time_of_day['late_night'] & occupancy['unoccupied'], lighting_intensity['off'])

# Rule 5: If it's early morning and the room is unoccupied, lighting should be off
rule5 = ctrl.Rule(time_of_day['early_morning'] & occupancy['unoccupied'], lighting_intensity['off'])

# Default rule for unoccupied states at all times of the day
default = ctrl.Rule(occupancy['unoccupied'], lighting_intensity['off'])

# Fuzzy control system
lighting_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, default])
lighting_simulation = ctrl.ControlSystemSimulation(lighting_control)



### Test the System with Example Data
We will now simulate different scenarios to observe how the system behaves.

In [14]:
# System test

# Example 1: 7 AM, Room Occupied
lighting_simulation.input['time_of_day'] = 7
lighting_simulation.input['occupancy'] = 1
lighting_simulation.compute()
print(f"Lighting intensity at 7 AM (Occupied): {lighting_simulation.output['lighting_intensity']:.2f}%")

# Example 2: 1 PM, Room Unoccupied
lighting_simulation.input['time_of_day'] = 13
lighting_simulation.input['occupancy'] = 0
lighting_simulation.compute()
print(f"Lighting intensity at 1 PM (Unoccupied): {lighting_simulation.output['lighting_intensity']:.2f}%")

# Example 3: 7 PM, Room Occupied
lighting_simulation.input['time_of_day'] = 19
lighting_simulation.input['occupancy'] = 1
lighting_simulation.compute()
print(f"Lighting intensity at 7 PM (Occupied): {lighting_simulation.output['lighting_intensity']:.2f}%")

# Example 4: 11 PM, Room Unoccupied
lighting_simulation.input['time_of_day'] = 23
lighting_simulation.input['occupancy'] = 0
lighting_simulation.compute()
print(f"Lighting intensity at 11 PM (Unoccupied): {lighting_simulation.output['lighting_intensity']:.2f}%")



Lighting intensity at 7 AM (Occupied): 50.00%
Lighting intensity at 1 PM (Unoccupied): 8.33%
Lighting intensity at 7 PM (Occupied): 75.00%
Lighting intensity at 11 PM (Unoccupied): 8.33%


### Potential Errors
- In case of sensor failure or missing data, the system might fail to compute the lighting intensity correctly. 
- Implementing default values or handling errors would ensure robust performance in the event of sensor issues.